<a href="https://colab.research.google.com/github/BaranikumarNagarajan/Spam_Navi-bayes/blob/main/Spam_NaiveBayes_Tried.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from textblob import TextBlob

In [ ]:
msg = pd.read_csv('Spamdata',sep = '\t', names = ["label", "message"])


In [ ]:
msg.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
msg.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
TextBlob('Hello world, how are you doing').tags

[('Hello', 'NNP'),
 ('world', 'NN'),
 ('how', 'WRB'),
 ('are', 'VBP'),
 ('you', 'PRP'),
 ('doing', 'VBG')]

In [ ]:
#lower case==>pos tagging ==> lemma ==> countvectorizer ==> tfidftransformer ==> MultiNB

def split_lemma(data):
  message = data.lower()
  words = TextBlob(message)
  tag_dict = {'J':'a','N':'n','V':'v','R':'r'}
  words_tags = [(w,tag_dict.get(pos[0],'n')) for w,pos in words.tags]
  return [wd.lemmatize(tag) for wd,tag in words_tags]

In [ ]:
msg.message.head().apply(split_lemma)

0    [go, until, jurong, point, crazy, .., availabl...
1                         [ok, lar, joke, wif, u, oni]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, do, n't, think, he, go, to, usf, he, ...
Name: message, dtype: object

In [ ]:
# checking purpose

msg.message.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: message, dtype: object

In [ ]:
count_transformer = CountVectorizer(analyzer = split_lemma).fit(msg['message'])

In [ ]:
print(len(count_transformer.vocabulary_))

8428


In [ ]:
count_vect = count_transformer.transform(msg['message'])

#pass the data to tf-idf fit method

tfidf_transformer = TfidfTransformer().fit(count_vect)

#transform

final_msg = tfidf_transformer.transform(count_vect)

print(final_msg.shape)

(5572, 8428)


In [ ]:
spam_model = MultinomialNB().fit(final_msg,msg['label'])

In [ ]:
#predict

sample = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'

predict_count = count_transformer.transform([sample])

predict_tfidf = tfidf_transformer.transform(predict_count)

print(predict_tfidf.shape)

(1, 8428)


In [ ]:
print(spam_model.predict(predict_tfidf)[0])

spam


In [ ]:
from sklearn.metrics import classification_report,f1_score,confusion_matrix,accuracy_score

In [ ]:
predictions = spam_model.predict(final_msg)

print('confusion matrix \n' , confusion_matrix(msg['label'],predictions))

confusion matrix 
 [[4825    0]
 [ 179  568]]


In [ ]:
print(classification_report(msg['label'],predictions))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      4825
        spam       1.00      0.76      0.86       747

    accuracy                           0.97      5572
   macro avg       0.98      0.88      0.92      5572
weighted avg       0.97      0.97      0.97      5572



In [ ]:
print(accuracy_score(msg['label'],predictions))

0.9678750897343862
